In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from datetime import date,timedelta,datetime
import re

In [2]:
def getComment(driver,formatted_date):
    pagenow = 1
    comment = []
    flag = True
    users = []
    dates = []
    acts = []
    merit = []
    while flag == True:
        dt = driver.find_elements(By.XPATH,"//tbody//tr//td[@valign='middle']//div[@class='smalltext']")
        content = driver.find_elements(By.XPATH,"//td[@class='td_headerandpost']//div[@class='post']")
        user = driver.find_elements(By.XPATH,"//td[@class='poster_info']//b//a")
        act = driver.find_elements(By.XPATH,"//td[@class='poster_info']//div[@class='smalltext']")
        content_list = []
        dt_list = []
        user_list = []
        act_list = []
        for i in range(min(len(content),len(user),len(act))):
            content_list.append(content[i].get_attribute('textContent'))
            user_list.append(user[i].get_attribute('textContent'))
            act_list.append(act[i].get_attribute('textContent'))
        for j in range(len(dt)):
            dt_list.append(dt[j].get_attribute('textContent'))
        indexes_to_remove2 = [index for index, item in enumerate(dt_list) if item.startswith('Merited')]
        for index2 in sorted(indexes_to_remove2, reverse=True):
            del dt_list[index2]
        indexes_to_remove = [index for index, item in enumerate(dt_list) if item[0].isdigit()]
        for index in sorted(indexes_to_remove, reverse=True):
            del content_list[index]
            del dt_list[index]
            del act_list[index]
            del user_list[index]
        
        for z in range(min(len(dt_list),len(user_list),len(content_list),len(act_list))):
            comment.append(content_list[z])
            users.append(user_list[z])
            dates.append(dt_list[z])
            match1 = re.search(r'Activity: (\d+)', act_list[z])
            activity_number = match1.group(1) if match1 else None
            acts.append(activity_number)
            match2 = re.search(r'Merit: (\d+)', act_list[z])
            merit_number = match2.group(1) if match2 else None
            merit.append(merit_number)
        page = driver.find_elements(By.XPATH,"//a[@class='navPages']")
        for k in range(len(page)):
            try:
                value = int(page[k].get_attribute('textContent'))
            except:
                value = 0
            if value == pagenow + 1:
                pagenow = pagenow+1
                page[k].click()
                time.sleep(2)
                break
        else:
            flag = False
    dates_to_remain = [index for index, item in enumerate(dates) if item.startswith('T')]
    comment = [comment[a] for a in dates_to_remain if a<len(comment)]
    dates = [dates[b] for b in dates_to_remain if b<len(dates)]
    users = [users[c] for c in dates_to_remain if c<len(users)]
    acts = [acts[d] for d in dates_to_remain if d<len(acts)]
    merit = [merit[e] for e in dates_to_remain if e<len(merit)]
    for w in range(len(dates)):
        if dates[w].startswith('T'):
            dates[w]  = formatted_date
    return comment,dates,users,acts,merit

In [3]:
def commentToday():
    path = r'./chromedriver.exe'
    cService = webdriver.ChromeService(executable_path=r'./chromedriver.exe')
    opt = Options()
    opt.add_argument("--headless") 
    driver = webdriver.Chrome(service = cService,options=opt)
    url = 'https://bitcointalk.org/index.php?board=1.0'
    driver.get(url)
    current_date = datetime.now()
    formatted_date = current_date.strftime('%B %d, %Y')
    df = pd.DataFrame(columns=['subject','replies','views','comment','date','user','activity','merit'])
    a = driver.find_elements(By.XPATH,"//td[@class='windowbg']//span/a")
    b = driver.find_elements(By.XPATH, "//td[@class='windowbg' and @valign='middle' and @width='4%' and @align='center']")
    c = driver.find_elements(By.XPATH, "//td[contains(@class, 'windowbg2') and contains(@class, 'lastpostcol')]//span[@class='smalltext']")
    url_list = []
    sub_list = []
    rep_list = []
    view_list = []
    date_list = []
    for j in range(min(len(a),len(b)//2)):
        url_list.append(a[j].get_attribute('href'))
        sub_list.append(a[j].get_attribute('textContent'))
        v1 = b[2*j].get_attribute('textContent').replace('\n', '').replace('\t', '')
        v2 = b[2*j+1].get_attribute('textContent').replace('\n', '').replace('\t', '')
        d1 = c[j].get_attribute('textContent').replace('\n', '').replace('\t', '')
        rep_list.append(v1)
        view_list.append(v2)
        date_list.append(d1)

    post_to_view = [index for index, item in enumerate(date_list) if item.startswith('T')]
    url_list = [url_list[k] for k in post_to_view if k<len(url_list)]

    for z in range(len(url_list)):
        current_url = driver.current_url
        driver.get(url_list[z])
        c,d,u,a,m = getComment(driver,formatted_date)
        for i in range(len(c)):
            ct = [sub_list[z],rep_list[z],view_list[z],c[i],d[i],u[i],a[i],m[i]]
            ctm = pd.DataFrame(ct).T
            ctm.columns=['subject','replies','views','comment','date','user','activity','merit']
            df = pd.concat([df,ctm])

        df = df.reset_index(drop=True)
        time.sleep(3)
        driver.get(current_url) 
    df_subject = df.drop_duplicates(subset=['subject'])[['subject','replies', 'views']]
    df_comment = df[['subject','comment','date','user']]
    df_user = df.drop_duplicates(subset=['user'])[['user','activity', 'merit']]
    return df_subject,df_comment,df_user

In [4]:
df_subject,df_comment,df_user = commentToday()
df_subject

,subject,replies,views
0,Bitcoin the HERO we never knew we needed,59,342
1,Bitcoin halving countdown,3,32
3,Bitcoin = Proof of Scam ?,19,130
4,El Salvador has become the first country to ma...,2454,33887
14,Craig Wright Satoshi Claim Case Update,63,733
20,Struggle now to invest in Bitcoin for a bright...,149,1249
27,Hodling or selling?,96,578
30,Why is Bitcoin the best asset for investment c...,98,688
34,BTC profit taking strategies?,134,1187
36,DO NOT BRAG ABOUT YOUR BITCOIN,128,968
